# Bac à sable RAG

Chargement de l'environnement

In [ ]:
import logging
import os

import s3fs
from langchain.schema.runnable.config import RunnableConfig
from langchain_core.prompts import PromptTemplate

from src.chain_building.build_chain import build_chain
from src.chain_building.build_chain_validator import build_chain_validator
from src.config import CHATBOT_TEMPLATE, EMB_MODEL_NAME
from src.db_building import load_retriever, load_vector_database
from src.model_building import build_llm_model

# Logging configuration
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s %(message)s",
    datefmt="%Y-%m-%d %I:%M:%S %p",
    level=logging.DEBUG,
)

# Remote file configuration
os.environ["MLFLOW_TRACKING_URI"] = "https://projet-llm-insee-open-data-mlflow.user.lab.sspcloud.fr/"
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": f"""https://{os.environ["AWS_S3_ENDPOINT"]}"""})

# PARAMETERS --------------------------------------

os.environ["UVICORN_TIMEOUT_KEEP_ALIVE"] = "0"

model = os.getenv("LLM_MODEL_NAME")
CHROMA_DB_LOCAL_DIRECTORY = "./data/chroma_db"
CLI_MESSAGE_SEPARATOR = f"{80*'-'} \n"
quantization = True
DEFAULT_MAX_NEW_TOKENS = 10
DEFAULT_MODEL_TEMPERATURE = 1
embedding = os.getenv("EMB_MODEL_NAME", EMB_MODEL_NAME)

model_id = "meta-llama/Llama-3.2-3B-Instruct"
LLM_MODEL = os.getenv("LLM_MODEL_NAME", "meta-llama/Llama-3.2-3B-Instruct")
LLM_MODEL = "microsoft/Phi-3.5-mini-instruct"
QUANTIZATION = os.getenv("QUANTIZATION", True)
MAX_NEW_TOKENS = int(os.getenv("MAX_NEW_TOKENS", DEFAULT_MAX_NEW_TOKENS))
MODEL_TEMPERATURE = int(os.getenv("MODEL_TEMPERATURE", DEFAULT_MODEL_TEMPERATURE))
RETURN_FULL_TEXT = os.getenv("RETURN_FULL_TEXT", True)
DO_SAMPLE = os.getenv("DO_SAMPLE", True)
DATABASE_RUN_ID = "32d4150a14fa40d49b9512e1f3ff9e8c"

## Ajoute un prompt

In [ ]:
from src.model_building.fetch_llm_model import cache_model_from_hf_hub

cache_model_from_hf_hub(
    model_id,
    s3_bucket=os.environ["S3_BUCKET"],
    s3_cache_dir="models/hf_hub",
    s3_endpoint=f'https://{os.environ["AWS_S3_ENDPOINT"]}',
)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    # token=os.getenv("HF_TOKEN"),
    use_fast=True,
    device_map="auto",
)

In [ ]:
# llm, tokenizer = build_llm_model(
#             model_name=model_id,
#             quantization_config=QUANTIZATION,
#             config=True,
#             token=os.getenv("HF_TOKEN"),
#             streaming=False,
#             generation_args={
#                 "max_new_tokens": 512,
#                 "return_full_text": RETURN_FULL_TEXT,
#                 "do_sample": DO_SAMPLE,
#                 "temperature": MODEL_TEMPERATURE
#             },
#     )

In [ ]:
db = load_vector_database(
    filesystem=fs,
    database_run_id=DATABASE_RUN_ID,
    # hard coded pour le moment
)

In [ ]:
retriever, vectorstore = load_retriever(
    emb_model_name=embedding,
    persist_directory=CHROMA_DB_LOCAL_DIRECTORY,
    vectorstore=db,
    retriever_params={"search_type": "similarity", "search_kwargs": {"k": 30}},
)

In [ ]:
len(db.get()["documents"])

## On repart de zéro

In [ ]:
def format_docs(docs: list):
    return "\n\n".join(
        [
            f"""
            Doc {i + 1}:\nTitle: {doc.metadata.get("Header 1")}\n
            Source: {doc.metadata.get("url")}\n
            Content:\n{doc.page_content}
            """
            for i, doc in enumerate(docs)
        ]
    )

In [ ]:
system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.
"""

question_instructions = """
Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
---

Voici la question à laquelle tu dois répondre :
Question: {question}

"""


RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    [
        {
            "role": "system",
            "content": system_instructions,
        },
        {"role": "user", "content": question_instructions},
    ],
    tokenize=False,
    add_generation_prompt=True,
)

prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)

In [ ]:
question1 = "Quelle est la définition du PIB ?"
question2 = "Où trouver les taux de chômage?"

documents_retriever_list1 = retriever.invoke(question1)
retrieved_docs_as_string1 = format_docs(documents_retriever_list1)
documents_retriever_list2 = retriever.invoke(question2)
retrieved_docs_as_string2 = format_docs(documents_retriever_list2)

prompt_injected1 = prompt.format(context=retrieved_docs_as_string1, question=question1)
prompt_injected2 = prompt.format(context=retrieved_docs_as_string2, question=question2)

In [ ]:
with open("prompt1.txt", "w") as file:
    file.write(prompt_injected1)

with open("prompt2.txt", "w") as file:
    file.write(prompt_injected2)

In [ ]:
print(prompt_injected1)

In [ ]:
with open("prompt1.txt") as file:
    prompt_injected1 = file.read()

with open("prompt2.txt") as file:
    prompt_injected2 = file.read()

In [ ]:
import torch
from vllm import LLM, SamplingParams

LLM_MODEL = model_id
MAX_NEW_TOKEN = 8192
TEMPERATURE = 0.2
REP_PENALTY = 1.1
TOP_P = 0.8

sampling_params = SamplingParams(
    max_tokens=MAX_NEW_TOKEN,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    repetition_penalty=REP_PENALTY,
)

llm = LLM(model=LLM_MODEL)

In [ ]:
outputs = llm.generate([prompt_injected1, prompt_injected2], sampling_params)

In [ ]:
responses = [outputs[i].outputs[0].text for i in range(len(outputs))]

In [ ]:
from IPython.display import Markdown, display


def printmd(string):
    display(Markdown(string))


printmd(responses[0])

In [ ]:
printmd(responses[1])

## Intégration à langchain

In [ ]:
with open("prompt1.txt") as file:
    prompt_injected1 = file.read()

with open("prompt2.txt") as file:
    prompt_injected2 = file.read()

In [ ]:
from langchain_community.llms import VLLM

LLM_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
MAX_NEW_TOKEN = 8192
TEMPERATURE = 0.2
REP_PENALTY = 1.1
TOP_P = 0.8

llm = VLLM(model=LLM_MODEL, max_new_tokens=MAX_NEW_TOKEN, top_p=TOP_P, temperature=TEMPERATURE, rep_penalty=REP_PENALTY)

In [ ]:
llm.invoke(prompt_injected1)

## Chaines chainlit

In [ ]:
system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes. 

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""

template = f"""
{system_instructions}

{question_instructions}
"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)
answer = rag_chain.invoke("Quelle est la définition du PIB ?")

In [ ]:
def printmd(string):
    display(Markdown(string))


printmd(answer)

In [ ]:
answer

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

for chunk in rag_chain.stream("Quelle est la définition du PIB?"):
    print(chunk, end="", flush=True)

In [ ]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(
    max_tokens=MAX_NEW_TOKEN,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    repetition_penalty=REP_PENALTY,
)

llm = LLM(model=LLM_MODEL)

In [ ]:
import pprint

from vllm import SamplingParams

sampling_params = SamplingParams(temperature=0.2, top_p=0.95)
outputs = llm.generate([prompt_injected], sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    # !r calls repr(), which prints a string inside quotes.
    print()
    print(f"Question: {question!r}")
    pprint.pprint(f"Generated text: {generated_text!r}")

In [ ]:
from langchain_community.llms import VLLM

llm = VLLM(
    model=model_id,
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=128,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
    # tensor_parallel_size=... # for distributed inference
)

print(llm("What is the capital of France ?"))

## Utilisation en direct de LLama

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

llama_direct = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", pad_token_id=tokenizer.eos_token_id
).to(device)


CHATBOT_TEMPLATE_bis = [
    {
        "role": "system",
        "content": system_instructions,
    },
    {"role": "user", "content": question_instructions.format(context=retrieved_docs_as_string, question=question)},
]

tokenized_chat = tokenizer.apply_chat_template(
    CHATBOT_TEMPLATE_bis, tokenize=True, add_generation_prompt=True, return_tensors="pt"
).to(device)

In [ ]:
print(tokenizer.decode(tokenized_chat[0]))

In [ ]:
outputs = llama_direct.generate(tokenized_chat, max_new_tokens=128000)

In [ ]:
print(tokenizer.decode(outputs[0]))

In [ ]:
response = tokenizer.decode(outputs[0]).split("tion du PIB?<|eot_id|><|start_header_id|>assistant<|end_header_id|>")[-1]

In [ ]:
def printmd(string):
    display(Markdown(string))


printmd(response)

In [ ]:
output  # .outputs[0].text

In [ ]:
CHATBOT_INSTRUCTION = """
En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.
La réponse doit être développée et citer ses sources.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.
"""

USER_INSTRUCTION = """
Voici le contexte sur lequel tu dois baser ta réponse :
Contexte:
{context}
---
Voici la question à laquelle tu dois répondre :
Question: {question}"""

CHATBOT_TEMPLATE = [
    {
        "role": "user",
        "content": """Tu es un assistant spécialisé dans la statistique publique.
    Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.
    Réponds en FRANCAIS UNIQUEMENT.""",
    },
    {"role": "assistant", "content": CHATBOT_INSTRUCTION},
    {"role": "user", "content": USER_INSTRUCTION},
]

RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(CHATBOT_TEMPLATE, tokenize=False, add_generation_prompt=True)
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)

In [ ]:
llm.generate(prompt)

In [ ]:
rag_chain_from_docs = RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"]))) | prompt

rag_chain_with_source = RunnableParallel({"context": retriever, "question": RunnablePassthrough()}).assign(
    answer=rag_chain_from_docs
)

question = rag_chain_with_source.invoke("Quelle est la définition du PIB ?")["answer"].to_string()

In [ ]:
question

On teste la question en standalone

In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.1-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    max_new_tokens=512,
    return_full_text=True,
    device_map="auto",
    do_sample=True,
    temperature=0.2,
)


messages = [
    {"role": "system", "content": "You assist French opendata specialists. You answer in French"},
    {"role": "user", "content": question},
]
outputs = pipe(
    messages,
    max_new_tokens=2000,
)

In [ ]:
answer_outside_chain = outputs[0]["generated_text"][-1]

In [ ]:
outputs

In [ ]:
CHATBOT_TEMPLATE = [
    {
        "role": "system",
        "content": """Tu es un assistant spécialisé dans la statistique publique.
    Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.
    Réponds en FRANCAIS UNIQUEMENT.
    
    En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.
    La réponse doit être développée et citer ses sources.

    Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.
    """,
    },
    {"role": "user", "content": question},
]

outputs2 = pipe(
    messages,
    max_new_tokens=2000,
)
answer_outside_chain2 = outputs2[0]["generated_text"][-1]

In [ ]:
outputs2[0][-1]["generated_text"][-1]

## Intégration dans un prompt

conclusion: c'est la galère, il vaut mieux utiliser vllm, cf. projet thomas 

In [ ]:
from src.config import CHATBOT_TEMPLATE as chatbottemplate

RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(chatbottemplate, tokenize=False, add_generation_prompt=True)
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)

In [ ]:
retrieved_documents = format_docs(retriever.invoke("Quelle est la définition du chômage ?"))

In [ ]:
example_messages = prompt.invoke(
    {"context": retrieved_documents, "question": "Quelle est la définition du PIB ?"}
).to_messages()

In [ ]:
rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt

retrieved_in_prompt = rag_chain.invoke("Quelle est la définition du PIB ?")

In [ ]:
messages = [
    {
        "role": "system",
        "content": """Tu es un assistant spécialisé dans la statistique publique.
    Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.
    Réponds en FRANCAIS UNIQUEMENT.
    En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.
    La réponse doit être développée et citer ses sources.

    Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.    
    """,
    },
    {"role": "system", "content": "You assist French opendata specialists. You answer in French"},
    {"role": "user", "content": question},
]

pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    max_new_tokens=512,
    return_full_text=True,
    device_map="auto",
    do_sample=True,
    temperature=0.2,
)

outputs = pipe(
    messages,
    max_new_tokens=2000,
)

In [ ]:
outputs[0]["generated_text"][-1]["content"]

In [ ]:
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(chatbottemplate, tokenize=False, add_generation_prompt=True)
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)

template = [
    {
        "role": "system",
        "content": """Tu es un assistant spécialisé dans la statistique publique.
    Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.
    Réponds en FRANCAIS UNIQUEMENT.
    En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.
    La réponse doit être développée et citer ses sources.

    Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.    
    """,
    },
    {
        "role": "user",
        "content": """
    Voici la question à laquelle tu dois répondre
    Question: {question}
    Context: {context}
    """,
    },
]

In [ ]:
prompt.invoke({"context": "filler context", "question": "filler question"}).to_messages()

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    max_new_tokens=512,
    return_full_text=True,
    device_map="auto",
    do_sample=True,
    temperature=0.2,
)

outputs = pipe(
    messages,
    max_new_tokens=2000,
)

hf = HuggingFacePipeline(pipeline=pipe, pad_token_id=pipe.model.config.eos_token_id)

In [ ]:
response_pipeline = hf.invoke(retrieved_in_prompt.to_string())

In [ ]:
response_pipeline

## Avec une chaine

In [ ]:
rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [ ]:
for chunk in rag_chain.stream("Quelle est la définition du PIB ?"):
    print(chunk, end="", flush=True)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"]))) | prompt | llm | StrOutputParser()
)

rag_chain_with_source = RunnableParallel({"context": retriever, "question": RunnablePassthrough()}).assign(
    answer=rag_chain_from_docs
)

In [ ]:
pib = rag_chain_with_source.invoke("Quelle est la définition du PIB ?")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    tokenizer=tokenizer,
)

messages = [
    {"role": "system", "content": "You assist French opendata specialists. You answer in French"},
    {"role": "user", "content": question},
]
outputs = pipe(
    messages,
    max_new_tokens=2000,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
from langchain_community.llms import HuggingFacePipeline

prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)


pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    tokenizer=tokenizer,
)

messages = [
    {"role": "system", "content": "You assist French opendata specialists. You answer in French"},
    {"role": "user", "content": "Quelle est la définition du PIB ?"},
]
outputs = pipe(
    messages,
    max_new_tokens=2000,
)
print(outputs[0]["generated_text"][-1])


# llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [ ]:
chain = build_chain(
    retriever=retriever,
    prompt=prompt,
    llm=llm,
    reranker=None,
)

In [ ]:
hf_pipeline = HuggingFacePipeline(pipeline=pipe)

In [ ]:
hf_pipeline.invoke("Donne moi les chiffres du chômage")

In [ ]:
from langchain_huggingface import HuggingFacePipeline

pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    max_new_tokens=512,
    return_full_text=True,
    device_map="auto",
    do_sample=True,
    temperature=0.2,
)

llm = HuggingFacePipeline(pipeline=pipe)
x = llm.invoke("je veux les chiffres du chômage en France")

In [ ]:
CHATBOT_INSTRUCTION = """
En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.
La réponse doit être développée et citer ses sources.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.
"""

USER_INSTRUCTION = """Voici le contexte sur lequel tu dois baser ta réponse :
Contexte:
{context}
---
Voici la question à laquelle tu dois répondre :
Question: {question}"""

CHATBOT_TEMPLATE = [
    {
        "role": "user",
        "content": """Tu es un assistant spécialisé dans la statistique publique.
    Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.
    Réponds en FRANCAIS UNIQUEMENT.""",
    },
    {"role": "assistant", "content": CHATBOT_INSTRUCTION},
    {"role": "user", "content": USER_INSTRUCTION},
]

In [ ]:
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(CHATBOT_TEMPLATE, tokenize=False, add_generation_prompt=True)
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)

In [ ]:
chain = RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"]))) | prompt | llm | StrOutputParser()

In [ ]:
llm, tokenizer = build_llm_model(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    quantization_config=QUANTIZATION,
    config=True,
    token=os.getenv("HF_TOKEN"),
    streaming=False,
    generation_args={
        "max_new_tokens": 100000,
        "max_length": 100000,
        "return_full_text": RETURN_FULL_TEXT,
        "do_sample": DO_SAMPLE,
        "temperature": MODEL_TEMPERATURE,
    },
)

In [ ]:
print(llm.invoke("quels sont les chiffres du chômage"))

In [ ]:
retriever.invoke("je veux les chiffres du chomage")

In [ ]:
db_docs = db.get()["documents"]
ndocs = f"Ma base de connaissance du site Insee comporte {len(db_docs)} documents"
ndocs

In [ ]:
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(CHATBOT_TEMPLATE, tokenize=False, add_generation_prompt=True)
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)

In [ ]:
validator = build_chain_validator(evaluator_llm=llm, tokenizer=tokenizer)

In [ ]:
from src.chain_building.build_chain import build_chain

In [ ]:
chain = build_chain(
    retriever=retriever,
    prompt=prompt,
    llm=llm,
    reranker="BM25",
)

In [ ]:
for s in chain.stream("Je veux les chiffres du chômage"):
    print(s)

Il y a un problème en ce moment sur la chaine, au niveau de la génération après retrieval. Je vais tester sur un exemple plus minime, à partir de 

* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines/#gpu-inference
* https://python.langchain.com/v0.1/docs/expression_language/interface/#async-stream

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
docs = retriever.invoke("je veux les chiffres du chômage")

In [ ]:
# await retrieval_chain.ainvoke("je veux les chiffres du chômage")

In [ ]:
retrieval_chain.batch(["je veux les chiffres du chômage", "quelle est la définition de l'inflation"])

In [ ]:
answer = retrieval_chain.invoke("je veux les chiffres du chômage")

In [ ]:
# chain.invoke("je veux les chiffres du chômage")
# await chain.ainvoke("je veux les chiffres du chômage")
for s in chain.stream("je veux les chiffres du chômage"):
    print(s)
# async for s in chain.astream("je veux les chiffres du chômage"):
#    print(s, end="", flush=True)

In [ ]:
def format_docs(docs: list):
    return "\n\n".join(
        [
            f"""
            Doc {i + 1}:\nTitle: {doc.metadata.get("Header 1")}\n
            Source: {doc.metadata.get("url")}\n
            Content:\n{doc.page_content}
            """
            for i, doc in enumerate(docs)
        ]
    )

In [ ]:
# stream = retrieval_chain.invoke("donne chiffres chomage")

llm.invoke("donne chiffres chomage")

In [ ]:
async for event in retrieval_chain.astream_events(
    "je veux les chiffres du chômage", version="v1", include_names=["Docs", "my_llm"]
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="|")
    elif kind in {"on_chat_model_start"}:
        print()
        print("Streaming LLM:")
    elif kind in {"on_chat_model_end"}:
        print()
        print("Done streaming LLM.")
    elif kind == "on_retriever_end":
        print("--")
        print("Retrieved the following documents:")
        print(event["data"]["output"]["documents"])
    elif kind == "on_tool_end":
        print(f"Ended tool: {event['name']}")
    else:
        pass

In [ ]:
import chainlit as cl

async for chunk in chain.astream(
    "je veux les chiffres du chomage",
    config=RunnableConfig(callbacks=[cl.AsyncLangchainCallbackHandler(stream_final_answer=True)]),
):
    if "answer" in chunk:
        await answer_msg.stream_token(chunk["answer"])
        generated_answer = chunk["answer"]

    if "context" in chunk:
        docs = chunk["context"]
        for doc in docs:
            sources.append(doc.metadata.get("url"))
            titles.append(doc.metadata.get("Header 1"))

In [ ]:
chain.invoke("Je veux les chiffres du chômage")